In [73]:
import pandas as pd
import numpy as np
import csv
import os
import sys
import matplotlib.pyplot as plt
import scipy
import scipy.stats as stat
import math

In [3]:
combined_maf = pd.read_csv('/Users/lmartin/Documents/2025/Rapid_Autopsy_Convergent_Evolution/Figure2/RA_combined_maf_dedup_updated_on_trees_022025.maf',sep='\t')
subclones_sig_genes = pd.read_csv('/Users/lmartin/Documents/2023/CDK4_mutsig_added_pats/on_subclones_v3/0/mutsig_results/outdir/sig_genes.txt',sep='\t')



In [8]:
subset=(subclones_sig_genes['p'] < 0.001)

In [11]:
coding_muts=['Missense_Mutation',
 'Nonsense_Mutation',
 'Start_Codon_SNP',
 'De_novo_Start_InFrame',
 'De_novo_Start_OutOfFrame',
 'Nonstop_Mutation',
 'Frame_Shift_Del',
 'Frame_Shift_Ins',
 'In_Frame_Del',
 'In_Frame_Ins']

coding_maf=combined_maf[combined_maf['Variant_Classification'].isin(coding_muts)]
len(coding_maf['Hugo_Symbol'].unique())

6790

In [12]:
pd.concat([
  coding_maf.groupby(["Hugo_Symbol", "Patient_ID"]).size().loc[subclones_sig_genes.loc[subset, "gene"]].groupby(level = 0).size().rename("n_mut_pats"),
  coding_maf.groupby(["Hugo_Symbol", "Tumor_Sample_Barcode"]).size().loc[subclones_sig_genes.loc[subset, "gene"]].groupby(level = 0).size().rename("n_mut_clones")],
  axis = 1
)

,n_mut_pats,n_mut_clones
Hugo_Symbol,,
CSAD,2,3
CTCF,2,4
CTNNA2,5,5
ESR1,6,10
HIST1H3B,2,3
IVL,4,6
KMT2C,5,16
KRAS,3,3
MEN1,1,2


## test with KMT2C

In [13]:
KMT2C_all_muts = coding_maf[coding_maf['Hugo_Symbol']=='KMT2C']
KMT2C_all_muts

,Patient_ID,Sample_ID,Sample_Alias,Hugo_Symbol,Chromosome,Start_position,Reference_Allele,Tumor_Seq_Allele,t_ref_count,t_alt_count,...,Cluster_Assignment,Allelic_CN_minor,Allelic_CN_major,preDP_ccf_mean,preDP_ccf_CI_low,preDP_ccf_CI_high,clust_ccf_mean,clust_ccf_CI_low,clust_ccf_CI_high,Tumor_Sample_Barcode
731,1547,6,NaN,KMT2C,7,151874772,C,T,180,6,...,15,NaN,NaN,0.09,0.04,0.14,0.05,0.05,0.06,1547_CL15
854,1547,10,NaN,KMT2C,7,151879260,C,A,184,12,...,14,NaN,NaN,0.18,0.10,0.27,0.06,0.06,0.06,1547_CL14
3714,1558,RA_1558_cfDNA_3,NaN,KMT2C,7,151879322,G,A,79,33,...,1,NaN,NaN,0.89,0.76,1.00,1.00,0.99,1.00,1558_CL1
3957,1558,1558_6,NaN,KMT2C,7,152012253,G,C,185,21,...,10,NaN,NaN,0.39,0.25,0.53,0.30,0.29,0.30,1558_CL10
4071,1326,RP-1700_BWES00001_v1_Exome_OnPrem,NaN,KMT2C,7,151896451,T,A,36,13,...,2,NaN,NaN,0.80,0.57,1.00,0.98,0.96,1.00,1326_CL2
4208,1326,RP-1700_T01027-3_v4_Exome_OnPrem,NaN,KMT2C,7,151945382,C,-,246,64,...,9,NaN,NaN,0.87,0.74,1.00,0.83,0.77,0.89,1326_CL9
4701,2389,1,NaN,KMT2C,7,151891593,G,A,101,46,...,5,NaN,NaN,0.77,0.62,0.92,0.88,0.86,0.90,2389_CL5
5037,2389,3,NaN,KMT2C,7,151859429,C,A,185,4,...,18,NaN,NaN,0.08,0.03,0.12,0.01,0.01,0.01,2389_CL18
5525,2389,4,NaN,KMT2C,7,151851222,G,C,158,10,...,4,NaN,NaN,0.22,0.11,0.36,0.16,0.15,0.17,2389_CL4
5526,2389,4,NaN,KMT2C,7,151878626,C,A,122,5,...,12,NaN,NaN,0.16,0.06,0.25,0.36,0.35,0.37,2389_CL12


In [16]:
## total number of mutations N0
N0 = len(KMT2C_all_muts)
## total number of mutated clones N_c
N_c = len( KMT2C_all_muts['Tumor_Sample_Barcode'].unique() )

In [97]:
N0

21

In [98]:
N_c

16

## start with one patient

In [19]:
RA_2389 = coding_maf[coding_maf['Patient_ID']==2389]

### for every clone, I am going to model the probability that KMT2C is in the clone or not
### then I'll tell it what the real distribution is and the sum is the score

In [265]:
## total number of mutations N0
N0 = len(KMT2C_all_muts)
## total number of mutated clones N_c
N_c = len( KMT2C_all_muts['Tumor_Sample_Barcode'].unique() )

RA_2389_clones = list(RA_2389['Tumor_Sample_Barcode'].unique() )
# total number of mutations in this patient
M = len(RA_2389)
 
# initialize a Pij matrix where you have a 1 if a clone was mutated and then the score
P_2389 =np.zeros( (len(RA_2389_clones), 2 ))

gene="KMT2C"

clone_idx=0
for clone in RA_2389_clones:
    clone_df = RA_2389[RA_2389['Tumor_Sample_Barcode'] == clone]
    mutations_in_clone = clone_df['Hugo_Symbol'].unique()
    Mij = len(list(mutations_in_clone))
    
    term_to_calc = (1 - ((Mij/M) * (N0/N_c)) ) ** N0
    
    # a given clone has no mutation in the gene
    P_2389[clone_idx, 0] = 1- term_to_calc
    
    # a given clone has a mutation in the gene
    P_2389[clone_idx, 1] = term_to_calc
    

    clone_idx +=1


In [266]:
## now every row sums to one

In [267]:
P_2389_prob_df = pd.DataFrame(P_2389, columns=['no_KMT2C', 'KMT2C'])
P_2389_prob_df.insert(0, 'clone', RA_2389_clones)
P_2389_prob_df

,clone,no_KMT2C,KMT2C
0,2389_CL16,0.780315,0.219685
1,2389_CL5,0.471755,0.528245
2,2389_CL3,0.534833,0.465167
3,2389_CL18,0.999999,0.000001
4,2389_CL14,0.584821,0.415179
5,2389_CL15,0.946495,0.053505
6,2389_CL1,0.186153,0.813847
7,2389_CL17,0.846040,0.153960
8,2389_CL6,0.566707,0.433293
9,2389_CL7,0.464263,0.535737


## patient score if no KMT2C mutation in any clone

In [268]:
P_2389_prob_df_no_KMT2C = P_2389_prob_df['no_KMT2C'].to_list()
logs_no_KMT2C = [ np.log(p) for p in P_2389_prob_df_no_KMT2C]
print( -np.sum(logs_no_KMT2C))

9.259727602330683


## patient score if a KMT2C mutation in all clones

In [269]:
P_2389_prob_df_KMT2C = P_2389_prob_df['KMT2C'].to_list()
logs_KMT2C = [ np.log(p) for p in P_2389_prob_df_KMT2C]
print( -np.sum(logs_KMT2C))

30.67738998828152


## test RA 1598, no KMT2C

In [271]:
RA_1598= coding_maf[coding_maf['Patient_ID']==1598]
RA_1598_clones = list(RA_1598['Tumor_Sample_Barcode'].unique() )

## total number of mutations N0
N0 = len(KMT2C_all_muts)
## total number of mutated clones N_c
N_c = len( KMT2C_all_muts['Tumor_Sample_Barcode'].unique() )

# total number of mutations in this patient
M = len(RA_1598)
# initialize a Pij matrix where you have a 1 if a clone was mutated 
P_1598 =np.zeros( (len(RA_1598_clones), 2 ))

gene="KMT2C"

clone_idx=0
for clone in RA_1598_clones:
    clone_df = RA_1598[RA_1598['Tumor_Sample_Barcode'] == clone]
    mutations_in_clone = clone_df['Hugo_Symbol'].unique()
    Mij = len(list(mutations_in_clone))
    term_to_calc = (1 - ((Mij/M) * (N0/N_c)) ) ** N0
    # a given clone has no mutation in the gene
    P_1598[clone_idx, 0] = 1- term_to_calc
    
    #  a given clone has a mutation in the gene
    P_1598[clone_idx, 1] = term_to_calc
    

    clone_idx +=1

In [272]:
P_1598_prob_df = pd.DataFrame(P_1598, columns=['no_KMT2C', 'KMT2C'])
P_1598_prob_df.insert(0, 'clone', RA_1598_clones)
P_1598_prob_df

,clone,no_KMT2C,KMT2C
0,1598_CL2,0.510435,4.895646e-01
1,1598_CL1,0.766279,2.337208e-01
2,1598_CL8,0.978552,2.144781e-02
3,1598_CL10,1.000000,3.977294e-09
4,1598_CL5,0.718135,2.818654e-01
5,1598_CL4,0.510435,4.895646e-01
6,1598_CL7,0.910726,8.927372e-02
7,1598_CL9,0.766279,2.337208e-01
8,1598_CL3,0.660635,3.393649e-01
9,1598_CL6,0.910726,8.927372e-02


## patient score if no KMT2C clones (truth)

In [133]:
P_1598_prob_df_no_KMT2C = P_1598_prob_df['no_KMT2C'].to_list()
logs_no_KMT2C_1598 = [ np.log(p) for p in P_1598_prob_df_no_KMT2C]
print( -np.sum(logs_no_KMT2C_1598))

2.831758513667644


## patient score if all KMT2C clones

In [140]:
P_1598_prob_df_KMT2C = P_1598_prob_df['KMT2C'].to_list()
logs_KMT2C_1598 = [ np.log(p) for p in P_1598_prob_df_KMT2C]
print( -np.sum(logs_KMT2C_1598))

34.69963201277903


## try for all patients for KMT2C

In [273]:
all_patients = list(combined_maf['Patient_ID'].unique())

In [290]:
all_probs_KMT2C = []

for patient in all_patients: 

    RA_maf = coding_maf[coding_maf['Patient_ID']==patient]
    RA_maf_clones = list(RA_maf['Tumor_Sample_Barcode'].unique() )
    
    ## total number of mutations N0
    N0 = len(KMT2C_all_muts)
    ## total number of mutated clones N_c
    N_c = len( KMT2C_all_muts['Tumor_Sample_Barcode'].unique() )
    
    # total number of mutations in this patient
    M = len(RA_maf)
    # initialize a Pij matrix where you have a 1 if a clone was mutated 
    P_pat =np.zeros( (len(RA_maf_clones), 2 ))
    
    gene="KMT2C"
    
    clone_idx=0
    for clone in RA_maf_clones:
        clone_df = RA_maf[RA_maf['Tumor_Sample_Barcode'] == clone]
        mutations_in_clone = clone_df['Hugo_Symbol'].unique()
        Mij = len(list(mutations_in_clone))
        
        term_to_calc = (1 - ((Mij/M) * (N0/N_c)) ) ** N0
        # probability that a given clone has a mutation in the gene
        P_pat[clone_idx, 1] = term_to_calc
        
        # probability that a given clone has no mutation in the gene
        P_pat[clone_idx, 0] = 1- term_to_calc
        clone_idx +=1

    P_prob_df = pd.DataFrame(P_pat, columns=['no_KMT2C', 'KMT2C'])
    P_prob_df.insert(0, 'clone', RA_maf_clones)
    all_probs_KMT2C.append(P_prob_df)
    

In [291]:
probs_1326 = all_probs[11]

## score if no KMT2C mutation in this patient

In [292]:
probs_1326_no_KMT2C = probs_1326['no_KMT2C'].to_list()
logs_no_KMT2C_1326 = [ np.log(p) for p in probs_1326_no_KMT2C]
print( -np.sum(logs_no_KMT2C_1326))

5.969068733986054


## real distribution

In [293]:
KMT2C_clones_RA1326= ['1326_CL2','1326_CL9']

In [294]:
P_1326_prob_df_KMT2C_clones = probs_1326[probs_1326['clone'].isin(KMT2C_clones_RA1326)]
P_1326_prob_df_non_KMT2C_clones = probs_1326[~probs_1326['clone'].isin(KMT2C_clones_RA1326)]

P_1326_prob_df_KMT2C_clones_list = P_1326_prob_df_KMT2C_clones['KMT2C'].to_list()
logs_real_KMT2C = [ -np.log(p) for p in P_1326_prob_df_KMT2C_clones_list]

P_1326_no_KMT2C_clones_list = P_1326_prob_df_non_KMT2C_clones['no_KMT2C'].to_list()
logs_real_no_KMT2C = [ -np.log(p) for p in P_1326_no_KMT2C_clones_list]


np.sum( np.concatenate((logs_real_KMT2C, logs_real_no_KMT2C)))

13.4568233637228

In [295]:
all_probs

[        clone  no_KMT2C         KMT2C
 0    1547_CL1  0.999841  1.589180e-04
 1    1547_CL8  0.660761  3.392387e-01
 2    1547_CL2  0.680027  3.199726e-01
 3   1547_CL14  1.000000  3.715205e-08
 4   1547_CL15  0.974996  2.500380e-02
 5   1547_CL12  0.572068  4.279320e-01
 6    1547_CL4  0.054017  9.459826e-01
 7    1547_CL6  0.105249  8.947513e-01
 8    1547_CL3  0.105249  8.947513e-01
 9   1547_CL11  0.775481  2.245195e-01
 10  1547_CL13  0.054017  9.459826e-01,
        clone  no_KMT2C         KMT2C
 0   1598_CL2  0.510435  4.895646e-01
 1   1598_CL1  0.766279  2.337208e-01
 2   1598_CL8  0.978552  2.144781e-02
 3  1598_CL10  1.000000  3.977294e-09
 4   1598_CL5  0.718135  2.818654e-01
 5   1598_CL4  0.510435  4.895646e-01
 6   1598_CL7  0.910726  8.927372e-02
 7   1598_CL9  0.766279  2.337208e-01
 8   1598_CL3  0.660635  3.393649e-01
 9   1598_CL6  0.910726  8.927372e-02,
       clone  no_KMT2C         KMT2C
 0  1078_CL1  0.999918  8.249165e-05
 1  1078_CL5  0.997347  2.652554e-03
 

In [280]:
all_probs_order = [ 1547, 1598, 1078, 1045, 1113, 1644, 2819, 2974, 1035,1534, 1558, 1326, 2389, 2542, 1002  ]
len(all_probs_order)

15

In [281]:
pats_with_no_KMT2C = [1598, 1078, 1045, 1113, 1644, 2819, 2974, 1035,1534,1002]
pats_with_no_KMT2C_idx =[1, 2, 3, 4, 5, 6, 7, 8, 9, 14]

In [282]:
probs_pats_no_KMT2C = []
probs_pats_no_KMT2C_scores = []
probs_pats_no_KMT2C_scores_dict = {}
for idx in pats_with_no_KMT2C_idx:
    score = 0
    probs_pats_no_KMT2C.append(all_probs[idx])
    pat_probs = all_probs[idx]
    pat_probs_no_KMT2C = pat_probs['no_KMT2C'].to_list()
    logs_no_KMT2C = [ np.log(p) for p in pat_probs_no_KMT2C]
    score = -np.sum(logs_no_KMT2C)
    
    probs_pats_no_KMT2C_scores.append(score)
    pat = all_probs_order[idx]
    probs_pats_no_KMT2C_scores_dict[pat] = score


In [283]:
probs_pats_no_KMT2C_scores_dict

{1598: 2.831758513667644,
 1078: 1.4099116376412282,
 1045: 7.422806015861028,
 1113: 1.8177880568251152,
 1644: 0.9490269803022182,
 2819: 5.1153659950758,
 2974: 2.7776425563567213,
 1035: 4.085005605081089,
 1534: 0.6526531549056974,
 1002: 21.021183275023688}

In [ ]:
## maybe apply monotonization for RA 1002?

In [192]:
pats_with_KMT2C_idx=[0, 10,11,12,13]

In [195]:
all_KMT2C_clones = list(KMT2C_all_muts['Tumor_Sample_Barcode'].unique())

In [288]:
probs_pats_with_KMT2C = []
probs_pats_KMT2C_scores = []
probs_pats_with_KMT2C_scores_dict = {}
for idx in pats_with_KMT2C_idx:
    #score=0
    running_score=[]
    pat_probs = all_probs[idx]
    all_clones_in_pat = list(pat_probs['clone'])
    for clone in all_clones_in_pat:
        pat_probs_clone = pat_probs[pat_probs['clone']==clone]
        if clone in all_KMT2C_clones:

            pat_probs_clone_KMT2C = pat_probs_clone['KMT2C'].to_list()
            logs_KMT2C = [ - np.log(p) for p in pat_probs_clone_KMT2C]
            running_score.extend(logs_KMT2C)
        else:
            pat_probs_no_KMT2C = pat_probs_clone['no_KMT2C'].to_list()
            logs_no_KMT2C = [ - np.log(p) for p in pat_probs_no_KMT2C]
            running_score.extend(logs_no_KMT2C)
            
    score = np.sum(running_score)
    
    probs_pats_with_KMT2C.append(pat_probs)
    probs_pats_KMT2C_scores.append(score)
    pat = all_probs_order[idx]
    probs_pats_with_KMT2C_scores_dict[pat] = score


In [289]:
probs_pats_with_KMT2C_scores_dict

{1547: 32.749645292558064,
 1558: 21.11796516113077,
 1326: 13.456823363722798,
 2389: 24.01724717124339,
 2542: 44.20458330060845}

In [230]:
probs_pats_KMT2C_scores

[32.749645292558064,
 21.11796516113077,
 13.456823363722798,
 24.01724717124339,
 44.20458330060845]

## test any gene

In [ ]:
def calculate_distributions(gene):

    gene_all_muts = coding_maf[coding_maf['Hugo_Symbol']==gene]
    all_probs = []
    
    for patient in all_patients: 
    
        RA_maf = coding_maf[coding_maf['Patient_ID']==patient]
        RA_maf_clones = list(RA_maf['Tumor_Sample_Barcode'].unique() )
        
        ## total number of mutations N0
        N0 = len(gene_all_muts)
        ## total number of mutated clones N_c
        N_c = len( gene_all_muts['Tumor_Sample_Barcode'].unique() )
        
        # total number of mutations in this patient
        M = len(RA_maf)
        # initialize a Pij matrix where you have a 1 if a clone was mutated 
        P_pat =np.zeros( (len(RA_maf_clones), 2 ))
        
        clone_idx=0
        for clone in RA_maf_clones:
            clone_df = RA_maf[RA_maf['Tumor_Sample_Barcode'] == clone]
            mutations_in_clone = clone_df['Hugo_Symbol'].unique()
            Mij = len(list(mutations_in_clone))
            term_to_calc = (1 - ((Mij/M) * (N0/N_c)) ) ** N0

            # probability that a given clone has no mutation in the gene
            P_pat[clone_idx, 0] = 1- term_to_calc
            
            
            # probability that a given clone has a mutation in the gene
            P_pat[clone_idx, 1] = term_to_calc
            clone_idx +=1

    
        P_prob_df = pd.DataFrame(P_pat, columns=['no_gene_mutations', 'gene_mutation'])
        P_prob_df.insert(0, 'clone', RA_maf_clones)
        all_probs.append(P_prob_df)
    return all_probs

In [264]:
esr1_Pm = calculate_distributions('ESR1')